In [1]:
import tifffile
import numpy as np
import napari

In [2]:
fix = tifffile.imread('//nrs.hhmi.org/liu/Jun/bigstream_test/raw_ref_t16_c3.tiff')
spacing = np.array([1., .27, .27])

print(fix.shape)
print(spacing) # same spacing for fix and mov

(341, 2304, 2304)
[1.   0.27 0.27]


In [6]:
from bigstream.level_set import foreground_segmentation
from scipy.ndimage import zoom, binary_closing, binary_dilation

# toss out some data, this is a low frequency easy thing to solve for
fix_skip = fix[::2, ::4, ::4]
fix_skip_spacing = spacing * [2, 4, 4]

# find the mask
fix_mask = foreground_segmentation(
    fix_skip, fix_skip_spacing,
    iterations=(40,8,2),
    shrink_factors=(4,2,1),
    smooth_sigmas=(8.,4.,2.),
    lambda2=20.,
    mask_smoothing=1,
    background=0,
)

# basic topological corrections and restore to full size
fix_mask = binary_closing(fix_mask, np.ones((5,5,5))).astype(np.uint8)
fix_mask = binary_dilation(fix_mask, np.ones((5,5,5))).astype(np.uint8)
fix_mask = zoom(fix_mask, np.array(fix.shape) / fix_skip.shape, order=0)

# save mask
tifffile.imwrite('./Mouse_emb_transformation/raw_ref_t16_mask.tiff', fix_mask, imagej=True, metadata={'axes':'ZYX'})

# load mask
fix_mask = tifffile.imread('./Mouse_emb_transformation/raw_ref_t16_mask.tiff') 


In [3]:
from bigstream.level_set import foreground_segmentation
from scipy.ndimage import zoom, binary_closing, binary_dilation

# load mask
fix_mask = tifffile.imread('./Mouse_emb_transformation/raw_ref_t16_mask.tiff') 


In [4]:
from bigstream.align import alignment_pipeline
from bigstream.transform import apply_transform

# define what type of alignment we want to do
affine_kwargs = {
    'alignment_spacing':2.0,
    'shrink_factors':(2,),
    'smooth_sigmas':(2.,),
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':700,
    },
}
steps = [('affine', affine_kwargs,),] # an option here is to deform: ('deform', deform_kwargs,) see deformable_align function in align.py and configure_irm.py for parameters


In [5]:
c = 3 # DAPI channel
T = [0,1,2,3,4,5,6,7]
for t in T:
    input_path = '//nrs.hhmi.org/liu/SpatialGenomicData/MouseETS5_E65/Round25-32/Time0000'+str(t)+'_Channel640,561,488,405_Seq000'+str(t)+'.ome.tiff'
    print(input_path)
    mat_path = './Mouse_emb_transformation/affine_ref_t16_t'+str(t+24).zfill(2)+'.mat'
    print(mat_path)    
    tif_path = './Mouse_emb_transformation/affine_ref_t16_t'+str(t+24).zfill(2)+'_c'+str(c)+'.tiff'
    print(tif_path)    

//nrs.hhmi.org/liu/SpatialGenomicData/MouseETS5_E65/Round25-32/Time00000_Channel640,561,488,405_Seq0000.ome.tiff
./Mouse_emb_transformation/affine_ref_t16_t24.mat
./Mouse_emb_transformation/affine_ref_t16_t24_c3.tiff
//nrs.hhmi.org/liu/SpatialGenomicData/MouseETS5_E65/Round25-32/Time00001_Channel640,561,488,405_Seq0001.ome.tiff
./Mouse_emb_transformation/affine_ref_t16_t25.mat
./Mouse_emb_transformation/affine_ref_t16_t25_c3.tiff
//nrs.hhmi.org/liu/SpatialGenomicData/MouseETS5_E65/Round25-32/Time00002_Channel640,561,488,405_Seq0002.ome.tiff
./Mouse_emb_transformation/affine_ref_t16_t26.mat
./Mouse_emb_transformation/affine_ref_t16_t26_c3.tiff
//nrs.hhmi.org/liu/SpatialGenomicData/MouseETS5_E65/Round25-32/Time00003_Channel640,561,488,405_Seq0003.ome.tiff
./Mouse_emb_transformation/affine_ref_t16_t27.mat
./Mouse_emb_transformation/affine_ref_t16_t27_c3.tiff
//nrs.hhmi.org/liu/SpatialGenomicData/MouseETS5_E65/Round25-32/Time00004_Channel640,561,488,405_Seq0004.ome.tiff
./Mouse_emb_transfo

In [5]:
from bigstream.align import alignment_pipeline
from bigstream.transform import apply_transform

# define what type of alignment we want to do
affine_kwargs = {
    'alignment_spacing':2.0,
    'shrink_factors':(2,),
    'smooth_sigmas':(2.,),
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':700,
    },
}
steps = [('affine', affine_kwargs,),] # an option here is to deform: ('deform', deform_kwargs,) see deformable_align function in align.py and configure_irm.py for parameters


# toss out some data, this is a low frequency easy thing to solve for
fix_skip = fix[::2, ::4, ::4]
fix_skip_spacing = spacing * [2, 4, 4]

In [12]:
from bigstream.align import alignment_pipeline
from bigstream.transform import apply_transform

# find transformation matrix using DAPI channel
c = 3 # DAPI channel
T = [0,1]
#T = [2,3,4,5,6,7]
for t in T:
    print(t)
    input_path = '//nrs.hhmi.org/liu/SpatialGenomicData/MouseETS5_E65/Round25-32/Time0000'+str(t)+'_Channel640,561,488,405_Seq000'+str(t)+'.ome.tiff'
    print(input_path)
    mov = tifffile.imread(input_path)[:, c, :, :].squeeze()
    
    mov_skip = mov[::2, ::4, ::4]
    mov_skip_spacing = spacing * [2, 4, 4]

    # find the mask for mov
    mov_mask = foreground_segmentation(
        mov_skip, mov_skip_spacing,
        iterations=(40,8,2),
        shrink_factors=(4,2,1),
        smooth_sigmas=(8.,4.,2.),
        lambda2=20.,
        mask_smoothing=1,
        background=0,
    )

    # basic topological corrections and restore to full size
    mov_mask = binary_closing(mov_mask, np.ones((5,5,5))).astype(np.uint8)
    mov_mask = binary_dilation(mov_mask, np.ones((5,5,5))).astype(np.uint8)
    mov_mask = zoom(mov_mask, np.array(fix.shape) / fix_skip.shape, order=0)
    
    # run the actual alignment
    affine = alignment_pipeline(
        fix, mov,
        spacing, spacing,
        steps=steps,
        fix_mask=fix_mask,
        mov_mask=mov_mask
    )
    # save result
    mat_path = './Mouse_emb_transformation/affine_ref_t16_t'+str(t+24).zfill(2)+'.mat'
    print(mat_path)
    np.savetxt(mat_path, affine)
    
    
    aligned = apply_transform(
        fix, mov, spacing, spacing,
        transform_list=[affine,], # 'affine' input here is the transformation matrix we got based on DAPI data
    )
    tif_path = './Mouse_emb_transformation/affine_ref_t16_t'+str(t+24).zfill(2)+'_c'+str(c)+'.tiff'
    print(tif_path)
    tifffile.imwrite(tif_path, aligned, imagej=True, metadata={'axes':'ZYX'})    

0
//nrs.hhmi.org/liu/SpatialGenomicData/MouseETS5_E65/Round25-32/Time00000_Channel640,561,488,405_Seq0000.ome.tiff
LEVEL:  0  ITERATION:  0  METRIC:  -1.2350136392970013
LEVEL:  0  ITERATION:  1  METRIC:  -1.3126553355920323
LEVEL:  0  ITERATION:  2  METRIC:  -1.4003903652018541
LEVEL:  0  ITERATION:  3  METRIC:  -1.4961139476845857
LEVEL:  0  ITERATION:  4  METRIC:  -1.5976595686764448
LEVEL:  0  ITERATION:  5  METRIC:  -1.7010524665092668
LEVEL:  0  ITERATION:  6  METRIC:  -1.799415818884232
LEVEL:  0  ITERATION:  7  METRIC:  -1.871290973754952
LEVEL:  0  ITERATION:  8  METRIC:  -1.8857237430996
LEVEL:  0  ITERATION:  9  METRIC:  -1.8905642998248036
LEVEL:  0  ITERATION:  10  METRIC:  -1.896252246656384
LEVEL:  0  ITERATION:  11  METRIC:  -1.9005830593611044
LEVEL:  0  ITERATION:  12  METRIC:  -1.9076572368309452
LEVEL:  0  ITERATION:  13  METRIC:  -1.9110223300854132
LEVEL:  0  ITERATION:  14  METRIC:  -1.9126836677589183
LEVEL:  0  ITERATION:  15  METRIC:  -1.9158150092851132
LEVEL

In [7]:
# apply the transformation matrix to all channels


spacing = np.array([1., .27, .27])

#T = [0]
T = [1,2,3,4,5,6,7]
C = [0,1,2]
for t in T:
    mat_path = './Mouse_emb_transformation/affine_ref_t16_t'+str(t+24).zfill(2)+'.mat'
    print(mat_path)
    affine = np.loadtxt(mat_path)
    input_path = '//nrs.hhmi.org/liu/SpatialGenomicData/MouseETS5_E65/Round25-32/Time0000'+str(t)+'_Channel640,561,488,405_Seq000'+str(t)+'.ome.tiff'
    print(input_path)
    for c in C:
        mov = tifffile.imread(input_path)[:, c, :, :].squeeze()
        
        aligned = apply_transform(
            fix, mov, spacing, spacing,
            transform_list=[affine,],
        )
        
        tif_path = './Mouse_emb_transformation/affine_ref_t16_t'+str(t+24).zfill(2)+'_c'+str(c)+'.tiff'
        print(tif_path)
        tifffile.imwrite(tif_path, aligned, imagej=True, metadata={'axes':'ZYX'})    
        

./Mouse_emb_transformation/affine_ref_t16_t25.mat
//nrs.hhmi.org/liu/SpatialGenomicData/MouseETS5_E65/Round25-32/Time00001_Channel640,561,488,405_Seq0001.ome.tiff
./Mouse_emb_transformation/affine_ref_t16_t25_c0.tiff
./Mouse_emb_transformation/affine_ref_t16_t25_c1.tiff
./Mouse_emb_transformation/affine_ref_t16_t25_c2.tiff
./Mouse_emb_transformation/affine_ref_t16_t26.mat
//nrs.hhmi.org/liu/SpatialGenomicData/MouseETS5_E65/Round25-32/Time00002_Channel640,561,488,405_Seq0002.ome.tiff
./Mouse_emb_transformation/affine_ref_t16_t26_c0.tiff
./Mouse_emb_transformation/affine_ref_t16_t26_c1.tiff
./Mouse_emb_transformation/affine_ref_t16_t26_c2.tiff
./Mouse_emb_transformation/affine_ref_t16_t27.mat
//nrs.hhmi.org/liu/SpatialGenomicData/MouseETS5_E65/Round25-32/Time00003_Channel640,561,488,405_Seq0003.ome.tiff
./Mouse_emb_transformation/affine_ref_t16_t27_c0.tiff
./Mouse_emb_transformation/affine_ref_t16_t27_c1.tiff
./Mouse_emb_transformation/affine_ref_t16_t27_c2.tiff
./Mouse_emb_transformatio